<a href="https://colab.research.google.com/github/mgnarag/binarization-autoencoder/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!ls drive/My\ Drive
file_path = "/content/drive/My Drive/Architectural_designs/one quadrant/"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf


from PIL import Image, ImageOps
Image.MAX_IMAGE_PIXELS = None
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow import keras

import time
import os
import cv2

In [ ]:
def crop(im):
    width, height = im.size
    data = []
    for j in range(0,int(height/n_size)): 
        for i in range(0,int(width/n_size)):
            im1 = im.crop((0 + (n_size*i), 0 + (n_size*j), n_size + (n_size*i), n_size + (n_size*j)))
            im1 = np.array(im1)
            im1 = im1.astype(np.float32)
            im1 = im1/255
            data.append(im1)
    return data

def normalize(arr):
    """
    Linear normalization
    http://en.wikipedia.org/wiki/Normalization_%28image_processing%29
    """
    arr = arr.astype('float')
    # Do not touch the alpha channel
    for i in range(3):
        minval = arr[...,i].min()
        maxval = arr[...,i].max()
        if minval != maxval:
            arr[...,i] -= minval
            arr[...,i] *= (255.0/(maxval-minval))
    return arr


In [ ]:
gt = Image.open(file_path+ '4 Testing/ground_truth.png').convert('L')
img_gt = cv2.imread(file_path+ '4 Testing/ground_truth.png')
gt = gt.crop((0, 0, 9856, 7168))
img_gt = img_gt[0:7168 , 0:9856]
gt = np.asarray(gt) > 128
gt = ~gt

filename = "1.png"
directory = file_path + '4 Testing/' + filename
#directory = 'test_output_v'+str(model)+'.png'
dirty = ImageOps.grayscale(Image.open(directory))
dirty = np.array(dirty)
dirty = Image.fromarray(normalize(dirty).astype('uint8'))

w_dirty, h_dirty = dirty.size


METRIC = []

for model in range(1,7):
    print('Model = ', model)
    from tensorflow import keras
    autoencoder = keras.models.load_model(file_path + '1 Models/autoencoder_'+str(model))
    if model == 1:
        n_size = 32
    if model == 2:
        n_size = 32
    if model == 3:
        n_size = 64
    if model == 4:
        n_size = 64
    if model == 5:
        n_size = 128
    if model == 6:
        n_size = 128   
  
    xx = int(w_dirty/n_size)
    final=[]

    for portion in range(0,xx):
        #print("current portion to clean:", str(portion))

        im1 = dirty.crop((n_size*portion, 0, (n_size*portion) + n_size, h_dirty-128))
        w1, h1 = im1.size
        w = int(w1/n_size)
        h = int(h1/n_size)

        neverbeforeseen = np.array(crop(im1))
        encoded_imgs = autoencoder.encoder(neverbeforeseen).numpy()
        decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()
        
        col = np.vstack((decoded_imgs[0],decoded_imgs[1]))
        for i in range(2,h):
            col = np.vstack((col,decoded_imgs[i]))
        
        y = np.where(col > 0.5,1,0) #round the values
        y = (y * 255).astype('uint8')
        if portion == 0:
            final = y
        if portion > 0:  
            final = np.hstack((final,y))

    final = np.squeeze(final)
    reconstructed = Image.fromarray(final)
    reconstructed.save(file_path + "5 Tested/output v" + str(model) + '.png') 

    output = Image.open(file_path + "5 Tested/output v" + str(model) + '.png').convert('L')  # convert to grayscale
    img_out = cv2.imread(file_path + "5 Tested/output v" + str(model) + '.png')

    output = output.crop((0, 0, 9856, 7168))
    img_out = img_out[0:7168 , 0:9856]

    psnr = cv2.PSNR(img_out, img_gt)
    # Convert to binary images
    output = np.asarray(output) > 128
    output = ~output

    tp = np.sum(output & gt)
    fp = np.sum(output & ~gt)
    fn = np.sum(~output & gt)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1score = (2*precision*recall)/(precision + recall)

    I_1 = output & gt
    O_1 = output | gt
    I = np.sum(output & gt)
    O = np.sum(output | gt)
    i_o_u = I/O
    METRIC.append([model, precision,recall,f1score,psnr,i_o_u])

METRIC = pd.DataFrame(METRIC) 

headers = ['Model','Precision', 'Recall', 'F1 Score', 'PSNR', 'IOU']
METRIC.columns = headers

METRIC.to_csv(file_path + 'Metric.csv') 

Model =  1
Model =  2
Model =  3
Model =  4
Model =  5
Model =  6


In [ ]:
gt = Image.open(file_path+ '4 Testing/ground_truth.png').convert('L')
img_gt = cv2.imread(file_path+ '4 Testing/ground_truth.png')
gt = gt.crop((0, 0, 9856, 7168))
img_gt = img_gt[0:7168 , 0:9856]
gt = np.asarray(gt) > 128
gt = ~gt

filename = "1.png"
directory = file_path + '4 Testing/' + filename
#directory = 'test_output_v'+str(model)+'.png'
dirty = ImageOps.grayscale(Image.open(directory))
dirty = np.array(dirty)
dirty = Image.fromarray(normalize(dirty).astype('uint8'))

w_dirty, h_dirty = dirty.size


METRIC = []

for model in range(1,3):
    print('Model = ', model)

    output = Image.open(file_path + "5 Tested/output v" + str(model) + '.png').convert('L')  # convert to grayscale
    img_out = cv2.imread(file_path + "5 Tested/output v" + str(model) + '.png')

    output = output.crop((0, 0, 9856, 7168))
    img_out = img_out[0:7168 , 0:9856]

    psnr = cv2.PSNR(img_out, img_gt)
    # Convert to binary images
    output = np.asarray(output) > 128
    output = ~output

    tp = np.sum(output & gt)
    fp = np.sum(output & ~gt)
    fn = np.sum(~output & gt)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1score = (2*precision*recall)/(precision + recall)

    I_1 = output & gt
    O_1 = output | gt
    I = np.sum(output & gt)
    O = np.sum(output | gt)
    i_o_u = I/O
    METRIC.append([precision,recall,f1score,psnr,i_o_u])

METRIC = pd.DataFrame(METRIC) 

headers = ['Precision', 'Recall', 'F1 Score', 'PSNR', 'IOU']
METRIC.columns = headers

METRIC.to_csv(file_path + 'Metric.csv') 

Model =  1
Model =  2
